In [17]:
import os

In [18]:
%pwd

'c:\\Users\\Hamza\\Desktop\\DL-kidney-disease-classification'

In [5]:
os.chdir("..")

In [6]:
%pwd

'c:\\Users\\Hamza\\Desktop\\DL-kidney-disease-classification'

In [19]:
import dagshub
import mlflow

dagshub.init(repo_owner='hamzaboughanim06', repo_name='DL-kidney-disease-classification', mlflow=True)



[2025-11-15 22:38:16,454: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/hamzaboughanim06/DL-kidney-disease-classification "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "hamzaboughanim06/DL-kidney-disease-classification"

[2025-11-15 22:38:16,469: INFO: helpers: Initialized MLflow to track repo "hamzaboughanim06/DL-kidney-disease-classification"]


Repository hamzaboughanim06/DL-kidney-disease-classification initialized!

[2025-11-15 22:38:16,478: INFO: helpers: Repository hamzaboughanim06/DL-kidney-disease-classification initialized!]


In [20]:
import tensorflow as tf


In [21]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [23]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [24]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/hamzaboughanim06/DL-kidney-disease-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [25]:
from pathlib import Path
import mlflow
import mlflow.keras
from mlflow.tracking import MlflowClient

from urllib.parse import urlparse

In [34]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            #
            if tracking_url_type_store != "file":

                # Register the model
                model_info = mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")


                # client = MlflowClient()
                # model_version = model_info.registered_model_version
                    
                #     # Set alias to the latest version
                # client.set_registered_model_alias(
                #     name="VGG16Model",
                #     alias="champion",
                #     version=model_version
                # )


            else:
                mlflow.keras.log_model(self.model, "model")

In [35]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-11-16 00:49:36,189: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-16 00:49:36,198: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-16 00:49:36,202: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 50s 5s/step - loss: 12.5041 - accuracy: 0.5396
[2025-11-16 00:50:27,393: INFO: common: json file saved at: scores.json]


2025/11/16 00:50:28 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-11-16 00:50:31,239: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Hamza\AppData\Local\Temp\tmp8to0093m\model\data\model\assets
[2025-11-16 00:50:32,299: INFO: builder_impl: Assets written to: C:\Users\Hamza\AppData\Local\Temp\tmp8to0093m\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/11/16 01:01:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.
